# Scrape Geschäfte
Ah I really hate this shitty API!

In [2]:
import urllib
import calendar
import requests
from pathlib import Path
import glob, os
import xml.etree.ElementTree as ET
import pandas as pd
import json
from dateutil import parser
import numpy as np

## 1. Load all Geschäfte
Store them as they are (XML)

In [3]:
url = 'https://parlzhcdws.cmicloud.ch/parlzh5/cdws/Index/GESCHAEFT/searchdetails?q=%s&l=de-CH'

year_from = 2001
year_to = 2023
print("Years: %s - %s" % (year_from, year_to))

for year in range(year_from, year_to):
    for i in range(0, 3):
        #print("%s %s" % (year, i))

        # First day
        d_start = '%s-%02d-01 00:00:00' % (year, i * 4 + 1)

        # Last day
        l = calendar.monthrange(year, (i + 1) * 4)[-1]
        d_end = '%s-%02d-%s 23:59:00' % (year, (i + 1) * 4, l)

        q = 'beginn_start >= "%s" and beginn_start <= "%s" sortBy beginn_start/sort.descending krnr/sort.descending' % (d_start, d_end)

        r = requests.get(url % urllib.parse.quote(q))
        with open(Path('../export/GESCHAEFT/%s_%s.xml' % (year, i)), 'wb') as f:
            f.write(r.content)


Years: 2001 - 2023


## 2. Create Document-DataFrame from XML
Logic: 
*  Search for `AR` (=Abstimmungsresultat). Then you will find ARs since 2020
* Search for `Abstimmungsresultat`. Then you will find older results
* Search for `Schlussabstimmung`

... yes... there is no other way... this API... let's say it nicely... no, better be quiet... ah, I hate it!

In [5]:
# Parse all XML and create Dataframe
ns = {
    'g': 'http://www.cmiag.ch/cdws/Geschaeft'
    }

records = []

# Open all xml-files
for f in glob.glob(str(Path('../export/GESCHAEFT/*.xml'))):
    tree = ET.parse(f)
    root = tree.getroot()
    
    # Map parents. Need it, to find parents
    parent_map = {c:p for p in tree.iter() for c in p}

    for g in root.findall('.//g:Ablaufschritt', ns):

        for d in g.findall('.//g:Dokument', ns):

            # If no version, skip it. Yes, there is unreferenced data in the database...
            versions = d.findall('.//g:Version', ns)
            if len(versions) == 0:
                continue

            geschaeft = parent_map[parent_map[g]]

            try:
                records.append({
                    'KRNr': geschaeft.find('g:KRNr', ns).text,
                    'VorlageNr': geschaeft.find('g:VorlageNr', ns).text,
                    'Titel': geschaeft.find('g:Titel', ns).text,
                    'Geschaeftsart': geschaeft.find('g:Geschaeftsart', ns).text,
                    'AblaufschrittTyp': g.find('g:AblaufschrittTyp', ns).text,
                    'DokumentTitel': d.find('g:Titel', ns).text,
                    'eDocumentFileName': d.find('g:eDocument', ns).attrib['FileName'],
                    'eDocumentID': d.find('g:eDocument', ns).attrib['ID'],
                    'Version': d.findall('.//g:Version', ns)[-1].attrib['Nr'],
                    'Sitzungsdatum': g.find('.//g:Sitzungsdatum/g:Start', ns).text,
                })

            except:
                print(d.findall('.//g:Version', ns))
                print(g.attrib['OBJ_GUID'])
                print(d.attrib['OBJ_GUID'])
                raise

df = pd.DataFrame(records)

df = df.sort_values('Sitzungsdatum')

# Add URL
df['url'] = df.apply(lambda row: 'https://parlzhcdws.cmicloud.ch/parlzh5/cdws/Files/{id}/{version}/pdf'.format(id=row['eDocumentID'], version=row['Version']), axis=1)

# Export only "Abstimmungen"
df_ar = df[
    (df.eDocumentFileName.str.startswith('AR')) |
    (df.eDocumentFileName.str.contains('Abstimmungsresultat', case=False)) |
    (df.eDocumentFileName.str.contains('Schlussabstimmung', case=False))
]

# Before Lockdown
df_ar = df_ar[df_ar['Sitzungsdatum'] < '2020-03-16']

df_ar.to_csv(Path('../export/abstimmungen.csv'), index=False)

len(df)


22037

## Download Abstimmungen (PDFs)

In [8]:
# Load file again
df_ar = pd.read_csv(Path('../export/abstimmungen.csv'))
df_ar['Sitzungsdatum'] = pd.to_datetime(df_ar['Sitzungsdatum'])

len(df_ar)

1118

In [10]:
# Download Documents
for i, row in df_ar.iterrows():

    fn = Path('../export/eDocuments/%s.pdf' % row['eDocumentID'])

    # Skip if already downloaded
    if fn.exists():
        continue

    r = requests.get(row['url'])

    if r.status_code != 200:
        print('File not found', row['url'])

    with open(fn, 'wb') as f:
        f.write(r.content)